# 🌳 **CO-FOREST** 🌳

##### **Autora: Patricia Hernando Fernández**

En proceso...

---------------------------
### **IMPORTS**

In [54]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import random
from sklearn.model_selection import train_test_split

### **CLASS**

In [55]:
class Co_Forest:

    def __init__(self, DL, U, n, sigma, n_class_types):
        """
        objetos de pandas, tanto L, como L_tags, como U
        DL -> Tupla (L, L_tags)
        """

        self.n = n
        self.sigma = sigma
        self.n_class_types = n_class_types

        self.ensemble = self.create_trees(DL) 
        self.fit(DL, U)


    def create_trees(self, DL):
        # keys: trees, values: L ignored during training
        ensemble = {}
        L, L_tags = DL

        for i in range(self.n):

            #cambiar valores tamaño a entrenar (está forzado para entrenar con pocos y que salte el error)
            X_train_h, X_ignore_h, y_train_h, y_ignore_h = train_test_split(L, L_tags, test_size=random.uniform(0.30, 0.50)) #CAMBIAR POR BOOTSTRAP
            h = DecisionTreeClassifier()
            h.fit(X_train_h.values, y_train_h.values)
            ensemble[h] = X_ignore_h.values #Cambiar a tuplas y hacer conjunto? hasheable para O(1)...

        return ensemble


    def fit(self, DL, U):

        L = DL[0]
        e_anterior = [0.5 for i in range(self.n)]
        W = [ min(0.1*len(L), 100) for i in range(self.n) ]

        t = 0
        new_data = True
        
        #ojito claves diccionario movidas de sitio (conc ensm)
        while new_data:
            t += 1
            i = 0

            for hi in self.ensemble.keys():

                x = U.sample().values[0]
                print("Error (todo L para los no entrenados con esos L): {}; Confidence (en una muestra de U): {}".format(self.calculate_e(hi, DL), self.confidence(hi, x, self.n_class_types)))
        
            break #QUITAR


    def calculate_e(self, hi, DL):

        sum_errores = total_dl_voted = 0
        DL_zip = zip(DL[0].values, DL[1].values) #Tuplas ([array features], int clase)

        for d in DL_zip:

            x, tag = d
            n_votes = n_hits = 0 

            for tree, ignored in self.ensemble.items():

                if tree != hi and x in ignored:

                    pred = hi.predict([x])[0]

                    if pred == tag:
                        n_hits += 1
                    n_votes +=1

            if (n_votes > 0):
                ex = 1 - (n_hits/n_votes)
                sum_errores += ex
                total_dl_voted += 1

        if total_dl_voted > 0:
            return sum_errores/total_dl_voted

        else:
            return 1 #No se ha votado, se repite -> CAMBIAR POR EXCEPCIÓN


    def confidence(self, hi, x, n_class_types):

        count = {}

        for i in range(n_class_types):
            count[i] = 0

        for tree in self.ensemble.keys():

            if tree != hi:
                prediction = tree.predict([x])[0]
                count[prediction] += 1

        max_agreement = max(count.values())
        confidence = max_agreement/(len(self.ensemble) -1)

        return confidence


### **DATA**

In [56]:
iris = load_iris()

data=pd.DataFrame({
    'sepal length':iris.data[:,0],
    'sepal width':iris.data[:,1],
    'petal length':iris.data[:,2],
    'petal width':iris.data[:,3],
    'species':iris.target
})

#data.head()
X=data[['sepal length', 'sepal width', 'petal length', 'petal width']] 
y=data['species']
n_class_types = len(iris.target_names)

#X_train, X_test, y_train, y_test
L, U, L_tags, U_tags = train_test_split(X, y, test_size=0.2)
DL = (L, L_tags)

### **TEST**

In [57]:
coforest = Co_Forest(DL, U, 10, 0.75, n_class_types)

Error (todo L para los no entrenados con esos L): 0.016666666666666666; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.025; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.05; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.025; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.016666666666666666; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.008333333333333333; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.03333333333333333; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.016666666666666666; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.025; Confidence (en una muestra de U): 1.0
Error (todo L para los no entrenados con esos L): 0.03333333333333333; C